In [1]:
import uproot
import yahist
import json
import glob
import numpy as np

from utils import make_table_1D, write_tool

In [2]:
def get_tgraph(name, root_file):
    # Get TGraph
    with uproot.open(root_file) as f:
        g = f.get(name)
    # Translate to a 1D hist with asymmetric error bars
    bin_edges = np.union1d(g._fX - g._fEXlow, g._fX + g._fEXhigh)
    bin_counts = g._fY
    h = yahist.Hist1D.from_bincounts(bin_counts, bins=bin_edges)
    h._errors_up = g._fEYhigh
    h._errors_down = g._fEYlow
    return h

In [3]:
def make_trigger_func(root_file_map, func_name):
    all_cpp = f"\nfloat {func_name}(float eta, float pt, int year) {{\n"
    for year, root_file in root_file_map.items():
        all_cpp += f"    // {root_file.split('/')[-1]}\n"
        with uproot.open(root_file) as f:
            # Get all histogram names
            hist_names = [k.decode("utf-8")[:-2] for k in f.keys() if "ZMassEta" in str(k)]
        # Extract eta bins
        eta_bins = []
        eta_overflow = False
        eta_underflow = False
        for n in hist_names:
            n = n.split("_")[0].split("ZMassEta")[-1]
            if "Lt" in n:
                eta_bins.append(float(n.split("Lt")[-1].replace("p", ".")))
                eta_underflow = True
            elif "Gt" in n:
                eta_bins.append(float(n.split("Gt")[-1].replace("p", ".")))
                eta_overflow = True
            else:
                eta_bins += [float(b.replace("p", ".")) for b in n.split("to")]
        eta_bins = sorted(list(set(eta_bins)))
        # Zip up data and MC hist names for ease of use
        hist_names = list(zip(
            [str(n) for n in hist_names if "_Data" in str(n)],
            [str(n) for n in hist_names if "_MC" in str(n)]
        ))
        # Make C++ lookup table
        cpp = ""
        for i, (data_name, mc_name) in enumerate(hist_names):
            # Just in case...
            if data_name.split("_")[0] != mc_name.split("_")[0]:
                print("ERROR: These two histograms belong to different eta bins!")
                print(data_name, mc_name)
                return
            if eta_underflow and i == 0:
                eta_condition = f"        if (fabs(eta) < {eta_bins[i]}) {{"
            elif eta_overflow and i == len(hist_names) - 1:
                eta_condition = f"        if (fabs(eta) >= {eta_bins[i-1]}) {{"
            else:
                eta_condition = f"        if (fabs(eta) < {eta_bins[i]} && fabs(eta) >= {eta_bins[i-1]}) {{"
            # Fetch data and MC histograms, and compute scale factors
            data_hist = get_tgraph(data_name, root_file)
            mc_hist = get_tgraph(mc_name, root_file)
            sf_hist = data_hist.divide(mc_hist)
            sf_hist._counts[~np.isfinite(sf_hist._counts)] = sf_hist._counts[np.isfinite(sf_hist._counts)][0]
            # Generate lookup table for the sf histogram
            sf_cpp = make_table_1D(
                sf_hist._counts, 
                sf_hist._edges, 
                x_name="pt", 
                x_abs=False, 
                x_type="float", 
                overflow=True
            )
            # Indent generated code
            sf_cpp = "\n".join(["        "+line for line in sf_cpp.split("\n")])
            # Append code
            cpp += "\n".join(
                [eta_condition, f"            // {data_name}/{mc_name}"]
                + sf_cpp.split("\n")[2:-3]
                + ["        }"]
            )
            if i < len(hist_names) - 1:
                cpp += "\n"
        # Append to function
        all_cpp += "\n".join([f"    if (year == {year}) {{"]+cpp.split("\n")+["    }", ""])
    all_cpp += "    return 0.0;\n"
    all_cpp += "}\n"
    return all_cpp

In [4]:
cpp = """#include "triggerScaleFactors.h"

float get2MuonTriggerEffSF(float pt, int year) {
    if (year == 2016) {
        return 0.99;
    }
    if (year == 2017) {
        if (pt >= 15 && pt < 40) return 0.97;
        if (pt >= 40 && pt < 55) return 0.995;
        if (pt >= 55 && pt < 70) return 0.96;
        if (pt >= 70) return 0.94;
    }
    if (year == 2018) {
        if (pt >= 15 && pt < 40) return 1.01;
        if (pt >= 40 && pt < 70) return 0.995;
        if (pt >= 70) return 0.98;
    }
    return 0.0;
}

float get2MuonTriggerEffSFErr(int year) {
    if (year == 2016) {
        return 0.01;
    }
    if (year == 2017) {
        return 0.02;
    }
    if (year == 2018) {
        return 0.01;
    }
    return 0.0;
}

float getMuonElecTriggerEffSF(float pt, int year) {
    if (year == 2016) {
        return 1.0;
    }
    if (year == 2017) {
        if (pt >= 15 && pt < 40) return 0.98;
        if (pt >= 40) return 0.99;
    }
    if (year == 2018) {
        if (pt >= 15 && pt < 25) return 0.98;
        if (pt >= 25) return 1.0;
    }
    return 0.0;
}

float getMuonElecTriggerEffSFErr(int year) {
    if (year == 2016) {
        return 0.01;
    }
    if (year == 2017) {
        return 0.01;
    }
    if (year == 2018) {
        return 0.01;
    }
    return 0.0;
}

float get2ElecTriggerEffSF(float pt, int year) {
    if (year == 2016) {
        if (pt >= 15 && pt < 25) return 0.98;
        if (pt >= 25) return 1.0;
    }
    if (year == 2017) {
        if (pt >= 15 && pt < 40) return 0.98;
        if (pt >= 40) return 1.0;
    }
    if (year == 2018) {
        if (pt >= 15 && pt < 25) return 0.98;
        if (pt >= 25) return 1.0;
    }
    return 0.0;
}

float get2ElecTriggerEffSFErr(int year) {
    if (year == 2016) {
        return 0.02;
    }
    if (year == 2017) {
        return 0.01;
    }
    if (year == 2018) {
        return 0.01;
    }
    return 0.0;
}
"""
with open("triggerScaleFactors.h", "w") as f_out:   
    f_out.write(f"#ifndef TRIGGERSCALEFACTORS_H\n#define TRIGGERSCALEFACTORS_H\n#include <math.h>\n\n")
    for line in [l[:-2]+";" for l in cpp.split("\n") if "get" in l]:
        f_out.write(line+"\n")
    f_out.write("\n#endif")

with open("triggerScaleFactors.cc", "w") as f_out:
    f_out.write(cpp)

In [5]:
hist_dir = "data/trigger_scalefactors"

el_root_files = {
    2016: f"{hist_dir}/Electron_Ele25WPTight_eff.root",
    2017: f"{hist_dir}/Electron_Ele32orEle35_eff.root",
    2018: f"{hist_dir}/Electron_Run2018_Ele32orEle35.root"
}

el_cpp = make_trigger_func(el_root_files, "getTauChannelElecTriggerSF")
write_tool(el_cpp, writemode="a", name="triggerScaleFactors")
err_cpp="""
float getTauChannelElecTriggerSFErr() {
    return 0.05;
}
"""
write_tool(err_cpp, writemode="a", name="triggerScaleFactors")

mu_root_files = {
    2016: f"{hist_dir}/Muon_Mu22OR_eta2p1_eff.root",
    2017: f"{hist_dir}/Muon_IsoMu24orIsoMu27_eff.root",
    2018: f"{hist_dir}/Muon_Run2018_IsoMu24orIsoMu27.root"
}
mu_cpp = make_trigger_func(mu_root_files, "getTauChannelMuonTriggerSF")
write_tool(mu_cpp, writemode="a", name="triggerScaleFactors")
err_cpp="""
float getTauChannelMuonTriggerSFErr() {
    return 0.05;
}
"""
write_tool(err_cpp, writemode="a", name="triggerScaleFactors")